In [31]:
# Imports
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [226]:
# reading in final dataframe
kahuna = pd.read_csv('../data/added_data/kahuna.csv')

-------------------------

## KMeans Clustering Preprocessing

In [227]:
X = kahuna.drop(columns=['Entity', 'Year'])
sc = StandardScaler()
X_sc = sc.fit_transform(X)

In [228]:
km_2 = KMeans(n_clusters=2, random_state=42)
km_2.fit(X_sc)

KMeans(n_clusters=2, random_state=42)

In [229]:
kahuna['clusterx2'] = km_2.labels_

In [230]:
km_4 = KMeans(n_clusters=4, random_state=42)
km_4.fit(X_sc)

KMeans(n_clusters=4, random_state=42)

In [231]:
kahuna['clusterx4'] = km_4.labels_

In [194]:
kahuna.to_csv('../data/new_cleaned/kahuna_clusters.csv')

-------------------------

### Modeling Classification (Entity, Year)

In [195]:
features_no_clust = kahuna.drop(columns=['clusterx3', 'clusterx10'])
features_clustx3 = kahuna.drop(columns=['clusterx10'])
features_clustx10 =  kahuna.drop(columns=['clusterx3'])
features_all = kahuna
features_list = [features_no_clust, features_clustx3, features_clustx10, features_all]

In [246]:
# drop Entity and Year column
X = kahuna.drop(columns=['Entity', 'Year'])

# set y = entity and label encode
y = kahuna['Entity']
le = LabelEncoder()
y = le.fit_transform(y)

# train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=24)

In [247]:
# scale data
ss = StandardScaler()

X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

#### Logistic Regression

In [248]:
class_list = []

In [249]:
# model for entity
logreg = LogisticRegression(random_state=42, max_iter=500)

logreg.fit(X_train_sc, y_train)
logiregr = ['logreg', logreg.score(X_train_sc, y_train), logreg.score(X_test_sc, y_test)]
class_list.append(logiregr)
logreg.score(X_train_sc, y_train), logreg.score(X_test_sc, y_test)

(0.9767441860465116, 0.9701492537313433)

In [200]:
# model for year
X = kahuna.drop(columns=['Entity', 'Year'])
y = kahuna['Year']
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

logreg = LogisticRegression(random_state=42, max_iter=500)

logreg.fit(X_train_sc, y_train)
logreg.score(X_train_sc, y_train), logreg.score(X_test_sc, y_test)

(0.10188261351052048, 0.05306799336650083)

#### RFC

In [250]:
# model for entity 
rfc = RandomForestClassifier(random_state=42)

rfc.fit(X_train, y_train)
ranfor = ['rfc', rfc.score(X_train, y_train), rfc.score(X_test, y_test)]
class_list.append(ranfor)
rfc.score(X_train, y_train), rfc.score(X_test, y_test)

(1.0, 0.9983416252072969)

In [201]:
# model for year
y = kahuna['Year']
le = LabelEncoder()
y = le.fit_transform(y)

rfc = RandomForestClassifier(random_state=42)

rfc.fit(X_train, y_train)
rfc.score(X_train, y_train), rfc.score(X_test, y_test)

(1.0, 0.004975124378109453)

In [251]:
class_table = pd.DataFrame(class_list, columns=['Model', 'Train', 'Test'])

In [254]:
class_table

Model     Train      Test
0  logreg  0.976744  0.970149
1     rfc  1.000000  0.998342

### Linear Regression for all features

In [202]:
# linear model for year
X = kahuna.drop(columns=['Entity', 'Year'])
y = kahuna['Year']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

ss = StandardScaler()

X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

lr = LinearRegression()

lr.fit(X_train_sc, y_train)
lr.score(X_train_sc, y_train), lr.score(X_test_sc, y_test)

(0.16630953852564334, 0.17256179720263654)

In [203]:
lr_list = []
for each in kahuna.drop(columns=['Entity', 'Year']).columns:
    X = kahuna.drop(columns=['Entity', 'Year', each])
    y = kahuna[each]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    # scale data
    ss = StandardScaler()

    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    lr = LinearRegression()
    
    lr.fit(X_train_sc, y_train)
    n = [each, lr.score(X_train_sc, y_train), lr.score(X_test_sc, y_test)]
    lr_list.append(n)
    print(f'lr of {each} = {lr.score(X_train_sc, y_train)}, {lr.score(X_test_sc, y_test)}')

lr of Deaths % = 0.8824878536561525, 0.866780478915891
lr of CO2 Emissions = 0.9972183718009016, 0.995067074482263
lr of CO2 Emissions Per Cap = 0.7627822936722819, 0.7505892651793498
lr of Food Emissions = 0.9413831688875416, 0.9336835942753924
lr of Total GHG = 0.9942845453794095, 0.9955870739772685
lr of Consumption of Ozone = 0.6280163563170563, 0.5187935135783976
lr of Shared CO2 Emissions = 0.9894351767847347, 0.9910456334705877
lr of Transport = 0.993710808224728, 0.9878625174981424
lr of Death_rate_ambient_ozone_pol = 0.6023111072752843, 0.6205686815895282
lr of Death_rate_household_air_pol = 0.9353663326692109, 0.9372624929794583
lr of Death_rate_all_causes = 0.7469854350522405, 0.722272187286938
lr of Death_actual_ozone = 1.0, 1.0
lr of Death_actual_particulate = 1.0, 1.0
lr of Death_actual_household = 0.9718776399046306, 0.9786157505191405
lr of Death_under5 = 1.0, 1.0
lr of Death_5-14 = 1.0, 1.0
lr of Death_50-69 = 1.0, 1.0
lr of Death_70+ = 1.0, 1.0
lr of Death_15-49 = 1.0

In [204]:
lr_table = pd.DataFrame(lr_list, columns=['Predictor', 'Train', 'Test'])

In [205]:
lr_table.T.drop(columns=[11, 12, 14, 15, 16, 17, 18]).T.sort_values(by='Test', ascending=False)

Predictor     Train      Test
27                                          clusterx2  0.997869  0.997752
24                                          clusterx3  0.997686  0.997308
4                                           Total GHG  0.994285  0.995587
1                                       CO2 Emissions  0.997218  0.995067
6                                Shared CO2 Emissions  0.989435  0.991046
7                                           Transport  0.993711  0.987863
13                             Death_actual_household  0.971878  0.978616
9                        Death_rate_household_air_pol  0.935366  0.937262
3                                      Food Emissions  0.941383  0.933684
0                                            Deaths %  0.882488   0.86678
20                                    Child Mortality  0.844264  0.846536
22                                                GDP  0.790329  0.798412
28                                          clusterx4  0.792846   0.76675
2                               CO2 Emissions Per Cap  0.762782  0.750589
19                                             Urban%  0.764928   0.74638
10                              Death_rate_all_causes  0.746985  0.722272
26  Incidence - Neoplasms - Sex: Both - Age: Age-s...  0.671196  0.675201
23                                        Forest area  0.689872  0.650717
8                        Death_rate_ambient_ozone_pol  0.602311  0.620569
25                                         clusterx10  0.605996  0.567926
5                                Consumption of Ozone  0.628016  0.518794
21                                         Population  0.261168  0.246907

In [222]:
cancer = pd.read_csv('../data/added_data/cancer-incidence.csv')

In [235]:
cancer.rename(columns={'Incidence - Neoplasms - Sex: Both - Age: Age-standardized (Rate)' : 'Cancer_Incidence'}, inplace=True)

In [236]:
cancer = cancer.drop(columns='Code')

KeyError: "['Code'] not found in axis"

In [237]:
cancer = cancer[cancer['Year']<=2014]

In [238]:
kahuna = kahuna.merge(cancer, on=['Entity', 'Year'])

In [240]:
kahuna.to_csv('../data/added_data/kahuna_clusters_cancer')

In [207]:
for each in kahuna.drop(columns=['Entity', 'Year', 'clusterx2', 'clusterx4', 'Death_rate_ambient_ozone_pol']).columns:
    X = kahuna.drop(columns=['Entity', 'Year', 'clusterx2', 'clusterx4', each])
    y = kahuna[each]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    # scale data
    ss = StandardScaler()

    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    lr = LinearRegression()
    
    lr_each.fit(X_train_sc, y_train)
    print(f'lr of {each} = {lr_each.score(X_train_sc, y_train)}, {lr_each.score(X_test_sc, y_test)}')

lr of Deaths % = 0.8716562191520321, 0.8567322448758011
lr of CO2 Emissions = 0.9968081264652657, 0.9945468498821548
lr of CO2 Emissions Per Cap = 0.754423805500311, 0.7469525334750731
lr of Food Emissions = 0.9407673782125073, 0.9316752211978517
lr of Total GHG = 0.9942444251942821, 0.9955435649402853
lr of Consumption of Ozone = 0.6124460486741606, 0.5395352564406828
lr of Shared CO2 Emissions = 0.9891531003527557, 0.9906501377849518
lr of Transport = 0.9890096438713155, 0.978879303040661
lr of Death_rate_household_air_pol = 0.9291674192039528, 0.9337958437122305
lr of Death_rate_all_causes = 0.7377652887033654, 0.709095667782081
lr of Death_actual_ozone = 1.0, 1.0
lr of Death_actual_particulate = 1.0, 1.0
lr of Death_actual_household = 0.969916848966614, 0.9762501015911581
lr of Death_under5 = 1.0, 1.0
lr of Death_5-14 = 1.0, 1.0
lr of Death_50-69 = 1.0, 1.0
lr of Death_70+ = 1.0, 1.0
lr of Death_15-49 = 1.0, 1.0
lr of Urban% = 0.7437850075570294, 0.729705080735542
lr of Child Morta

In [255]:
kahuna

Entity  Year   Deaths %  CO2 Emissions  CO2 Emissions Per Cap  \
0      Albania  1991  17.613636        4228256                 1.2889   
1      Albania  1995  16.789055        2066496                 0.6638   
2      Albania  1996  16.588451        1993216                 0.6432   
3      Albania  1997  15.081361        1531552                 0.4941   
4      Albania  1998  15.965796        1744064                 0.5607   
...        ...   ...        ...            ...                    ...   
2404  Zimbabwe  2010   9.450169        7878316                 0.6205   
2405  Zimbabwe  2011  10.053401        9743983                 0.7557   
2406  Zimbabwe  2012  10.455091        7882889                 0.6011   
2407  Zimbabwe  2013  10.691584       11836254                 0.8866   
2408  Zimbabwe  2014  10.858493       11905687                 0.8763   

      Food Emissions   Total GHG  Consumption of Ozone  Shared CO2 Emissions  \
0          4011691.0   9030000.0                  0.00                  0.02   
1          4861321.1   7480000.0                 43.40                  0.01   
2          4569702.9   7280000.0                 43.20                  0.01   
3          4431656.7   6510000.0                 45.20                  0.01   
4          4344061.6   6810000.0                 49.60                  0.01   
...              ...         ...                   ...                   ...   
2404      55244889.2  66540000.0                 29.30                  0.02   
2405      53985614.6  67420000.0                 22.22                  0.03   
2406      54211027.7  67630000.0                 16.20                  0.02   
2407      54321218.5  67550000.0                 15.76                  0.03   
2408      51674911.7  66100000.0                 13.32                  0.03   

      Transport  ...   Death_70+  Death_15-49  Urban%  Child Mortality  \
0      500000.0  ...  512.126572    60.714220  36.700             3.93   
1      600000.0  ...  469.406670    49.692924  38.911             3.39   
2      600000.0  ...  491.357876    52.069689  39.473             3.26   
3      500000.0  ...  521.197486    57.988035  40.035             3.13   
4      700000.0  ...  547.315668    64.973478  40.601             2.99   
...         ...  ...         ...          ...     ...              ...   
2404  1300000.0  ...  684.000846   253.211851  33.196             8.76   
2405  2300000.0  ...  687.415400   252.443341  33.015             8.14   
2406  2600000.0  ...  711.735517   258.678699  32.834             7.38   
2407  2900000.0  ...  740.153224   266.910947  32.654             6.78   
2408  2600000.0  ...  766.192922   280.271101  32.504             6.44   

      Population          GDP  Forest area  clusterx2  clusterx4  \
0     119.225912  3496.580246     786850.0          0          2   
1     116.342482  4471.871070     779050.0          0          2   
2     115.621642  4909.228105     777100.0          0          2   
3     114.900766  4400.577827     775150.0          0          2   
4     114.179927  4819.387534     773200.0          0          2   
...          ...          ...          ...        ...        ...   
2404   32.823389  2458.220626   17905280.0          0          2   
2405   33.331583  2764.338916   17859210.0          0          2   
2406   33.902414  3170.726522   17813140.0          0          2   
2407   34.510477  3176.829330   17767070.0          0          2   
2408   35.121391  3195.767970   17721000.0          0          2   

      Cancer_Incidence  
0           190.046659  
1           183.178817  
2           187.578978  
3           191.441640  
4           195.033589  
...                ...  
2404        224.459639  
2405        216.792521  
2406        210.917010  
2407        206.449353  
2408        203.151575  

[2409 rows x 29 columns]

In [256]:
kahuna.columns

Index(['Entity', 'Year', 'Deaths %', 'CO2 Emissions', 'CO2 Emissions Per Cap',
       'Food Emissions', 'Total GHG', 'Consumption of Ozone',
       'Shared CO2 Emissions', 'Transport', 'Death_rate_ambient_ozone_pol',
       'Death_rate_household_air_pol', 'Death_rate_all_causes',
       'Death_actual_ozone', 'Death_actual_particulate',
       'Death_actual_household', 'Death_under5', 'Death_5-14', 'Death_50-69',
       'Death_70+', 'Death_15-49', 'Urban%', 'Child Mortality', 'Population',
       'GDP', 'Forest area', 'clusterx2', 'clusterx4', 'Cancer_Incidence'],
      dtype='object')